## Import Libraries

In [1]:
import os
import openai
import json
import pandas as pd
import logging
from datetime import datetime
import time

In [2]:
# Remove column width to ensure that all characters are displayed
pd.set_option("display.max_colwidth", None)

## Define Function to Request Chat GPT to Write Advert

## Task 1: Write a Script for an Advert Promoting Product X

### Define Function to Make Request to ChatGPT

In [3]:
def chat_advert(product):
    
    openai.api_key = os.getenv('OPEN_AI_API_KEY')
    
    # Create log file
    logging.basicConfig(filename=f'{datetime.now().strftime("%Y%m%d%H%M%S")}_chatgpt.log', encoding='utf-8', level=logging.DEBUG, format='%(asctime)s %(message)s')

    try:
        
        response = openai.ChatCompletion.create(model="gpt-4", 
                                                messages=[{"role":"user","content":f"Write a script for an advert promoting {product}"}], temperature=0.7)
        logging.info(f"Request successful") 
        return response


    except openai.error.RateLimitError as e:
        
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 30
        logging.info(f"Rate limit exceeded. Retrying in {retry_time} seconds...")
        print(f"Rate limit exceeded. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        return

    except openai.error.ServiceUnavailableError as e:
        retry_time = 10  # Adjust the retry time as needed
        logging.info(f"Service is unavailable. Retrying in {retry_time} seconds...")
        print(f"Service is unavailable. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        return

    except openai.error.APIError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 30
        logging.info(f"API error occurred. Retrying in {retry_time} seconds...")
        print(f"API error occurred. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        return

    except OSError as e:
        retry_time = 5  # Adjust the retry time as needed
        logging.info(f"Connection error occurred: {e}. Retrying in {retry_time} seconds...")
        print(f"Connection error occurred: {e}. Retrying in {retry_time} seconds...")      
        time.sleep(retry_time)
        return

### Products

In [4]:
products = ['beer','chocolate','ice cream','protein powder','a weight loss programme','a lawnmower','a car','a diy store',
            'a supermarket','a clothes shop','furniture polish','a washing machine','dishwasher tablets','a vacuum cleaner',
            'candles','bubble bath','curtains','electric drills','nappies','a science museum','an art gallery',
            'a bookshop','a games console','a social network','a yoga class','a weightlifting class','a golf club','therapy']

In [5]:
print("Number of Products")
len(products)

Number of Products


28

### Run Request to Chat-GPT (GPT-4-Turbo)

In [22]:
# Query the Open AI API using hte prompt "Write a script for an advert promoting X"
# For each product, run prompt X times - generate a sufficiently large dataset
# Append response to list
responses = []

for iteration in list(range(0,10)):
    for product in products:
        response = chat_advert(product)
        responses.append(response)

In [23]:
len(responses)/28

10.0

In [24]:
responses[1]

<OpenAIObject chat.completion id=chatcmpl-7nAVphaPMbDptcHNEDP7ZU9Jcyb7c at 0x7fb8857047c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "[Opening scene: A close-up shot of a luscious, glossy chocolate bar, slowly melting under a stream of warm golden light.]\n\nNarrator (in a warm, inviting voice): \"Indulge in the exquisite world of pure indulgence...welcome to ChocoLux - luxurious chocolate crafted to perfection.\"\n\n[Scene shifts to a cacao tree with ripe cacao pods. A hand gently picks a pod.]\n\nNarrator: \"Sourced from the finest cacao trees, each pod is handpicked to ensure only the highest quality for our chocolate.\"\n\n[Scene: Workers carefully roasting and grinding the cacao beans.]\n\nNarrator: \"Roasted to perfection and ground delicately, we strive to preserve the rich, authentic taste of cacao.\"\n\n[Scene: Liquid chocolate being poured into moulds.]\n\nNarrator: \"Moulded with love and care, every b

In [25]:
# Update the number of times the products list is replicated with the number 
products_multiplied = []

for i in list(range(0,10)):
    products_multiplied = products_multiplied+products

In [26]:
# This code needs to be updated with a new file name to ensure that previous responses are not overwritten

# Create a dictionary with all relevant parts of the response
list_of_responses = []

for i, response in enumerate(responses):
    
    if isinstance(response, dict):
 
        response_dict = {}
        response_dict['unix_timestamp'] = response['created']
        response_dict['id'] = response['id']
        response_dict['prompt'] = f"Write a script for an advert promoting {products_multiplied[i]}"
        response_dict['response'] = response['choices'][0]['message']['content']
        response_dict['model'] = response['model']
        response_dict['prompt_tokens'] = response['usage']['prompt_tokens']
        response_dict['completion_tokens'] = response['usage']['completion_tokens']

        list_of_responses.append(response_dict)
    
    else:
        
        continue

# Convert dictionary to json
response_json = json.dumps(list_of_responses)

# Dump the json file 
# Update the file name so nothing is overwritten
out_file = open("gpt4_responses_bulk_part_3.json", "w")
json.dump(response_json,out_file)